# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Youtube video analysis
* This notebook explains how to use Gemini to understand images in multimodality features of Gemini. This code shows how to use Gemini to analyze a Youtube videos with the feature.
* The youtube video that is used in this demo is "https://www.youtube.com/watch?v=rwF-X5STYks", the usage of the videos is just for the educational purpose.
* The video owner is KI Campus, please contact them to get permission if you want to use it.
* Don't use this Youtube video for the other purpose.
* Refer to the link for more information about the Gemini
 * ***https://cloud.google.com/vertex-ai/generative-ai/docs/learn/overview***

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
* Install pytube to download the Youtube video
  * https://github.com/pytube/pytube
  * pip install pytube

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 7.9 MB/s eta 0:00:00


In [2]:
%pip install -q -U pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 622.6 kB/s eta 0:00:00


In [3]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [4]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [5]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


## Download the Youtube video to anlyze.
* The youtube video that is used in this demo is "https://www.youtube.com/watch?v=rwF-X5STYks", the usage of the videos is just for the educational purpose.
* The video owner is KI Campus, please contact them to get permission if you want to use it.

In [6]:
from pytube import YouTube

# Download youtube video to your local drive in colab.
YouTube('https://www.youtube.com/watch?v=rwF-X5STYks').streams.first().download()


'/content/Generative AI explained in 2 minutes.mp4'

In [7]:
if "google.colab" in sys.modules:

  import moviepy.editor
  moviepy.editor.ipython_display("Generative AI explained in 2 minutes.mp4", maxduration = 130)

# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [8]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.

In [9]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Encoding function for multimodality

In [10]:
import base64

def get_encoded_content(location_type, location, mime_type ):
  """
  Get the encoded content object.

  location_type :
    The type of the location. ( local or GCS )
  location :
    The file location of the content.
  mime_type :
    The mime type of the content.

  Returns:
    The encoded content object.

  """

  content_obj = None

  if location_type == "local":
    with open(location, 'rb') as f:
      raw_obj = base64.b64encode(f.read()).decode('utf-8')
      content_obj = Part.from_data(data=base64.b64decode(raw_obj), mime_type=mime_type)

  elif location_type == "GCS":
        content_obj = Part.from_uri(location, mime_type=mime_type)
  else:
    raise ValueError("Invalid location type.")

  return content_obj

### Function to get the response

In [11]:
def generate(content_obj, query:str):
    """
    Generate a response from the model.

    content_obj :
      encoded object being analyzed in the process
    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [content_obj, query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

## Run example

In [12]:

from time import perf_counter

t1_start = perf_counter()

# When using local storage for the file location.
location_type = "local"
mime_type = "video/mp4"

repository_root = "."
file_path = "/Generative AI explained in 2 minutes.mp4"
location = repository_root + file_path

content_obj = get_encoded_content(location_type, location, mime_type )

prompt = """
You are a helpful assistant that summarizes the contents of the video.
1. Summarize the video contents in detail in 3 categories
2. Explain what's the key points of the contents
"""

outcome = generate(content_obj, prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


# When using Google Cloud Storage for the location type.

# location_type = "GCS"
# mime_type = "video/mp4"
# repository_root = "gs://bucket_name"
# file_path = "/Generative AI explained in 2 minutes.mp4"
# location = repository_root + file_path

Time : 31.019173418000037 seconds




Okay, here's a summary of the video about generative AI, categorized for clarity:

**1. Definition and Function:**

* **Generative AI** refers to the use of artificial intelligence to create new content such as texts, images, videos, music, or even voices.
* **Prompts:** To generate content, you need to provide instructions to the AI, called "prompts," which describe what you want the AI to create.
* **Data Training:** Generative AI tools rely on vast amounts of data, which they analyze to identify patterns and similarities. This data is essential for their ability to create new content.

**2. Applications and Examples:**

* **Creative Content:** Generative AI can be used to create short stories, paintings, code, musical compositions, and other forms of creative expression.
* **Information Processing:** Generative AI can summarize complex information and generate diverse ideas quickly.
* **Learning Partner:**  Generative AI can serve as a valuable learning partner, providing assistance in writing and research.

**3. Potential Risks and Considerations:**

* **Deep Fakes:** Generative AI can be misused to create highly convincing fabricated images and videos, posing challenges for distinguishing truth from falsehood.
* **Hallucinations:** Generative AI can produce outputs that sound correct but are factually inaccurate. This is known as "hallucination" and requires careful evaluation of AI-generated information.
* **Human Responsibility:**  While generative AI can be a powerful tool, it's crucial for humans to use these tools responsibly, providing accurate prompts and verifying the authenticity of AI-generated content.

**Key Points:**

* Generative AI is a rapidly evolving field with the potential to revolutionize content creation and information processing.
* The quality of the output depends on the quality of the training data and the clarity of prompts provided by the user.
* Recognizing the potential for misuse, such as deep fakes and hallucinations, is crucial for responsible use of generative AI.
* Humans must remain active participants in the process, utilizing AI as a tool while maintaining critical judgment and responsibility for the accuracy and ethical implications of the generated content.

Let me know if you have any other questions! 
